In [ ]:
# Notes - landscan will be the reference grid

In [ ]:
!pip install rasterio
!pip install pyproj

In [1]:
import os

import rasterio
from rasterio.windows import Window, from_bounds
from rasterio.warp import Resampling
from rasterio.vrt import WarpedVRT
from rasterio import shutil as rio_shutil

In [2]:
from pyproj import Transformer

# sanity check - coordinates to check
# first coord is in Yangon, second is rural, the rest are randomly selected from the survey
coords = [(96.199379, 16.871311), (96.103347, 18.613386), (97.233770, 24.260900), (95.419190, 17.681720), (96.163024, 16.774673), (95.404810, 16.286770)]

# given path to a tif file and a list of lon-lat coordinates
# this returns the values at those coordinates
def check_vals_coord(path, coord_lst):
    with rasterio.open(path) as rds:
        # convert coordinate to raster projection
        transformer = Transformer.from_crs(str(rds.crs), rds.crs, always_xy=True)
        pts = [transformer.transform(tup[0], tup[1]) for tup in coord_lst]
        # get value from grid
        vals = [list(rds.sample([e]))[0][0] for e in pts]
    return vals

In [10]:
dirpath = 'C:/Users/sun-joo.lee/Documents/0_git_repos/Myanmar_Survey_ML/data/geo/landscan/'
infile = 'global/landscan-global-2022.tif'
outfile = 'clipped/landscan_mm17.tif'

In [11]:
# bounding box coordinates for myanmar
mm_bbox = [91.5, 9.0, 101.5, 29.0]

In [12]:
# get vrt options from input file

inpath = dirpath + infile
with rasterio.open(inpath) as src:
    win = from_bounds(*mm_bbox, src.transform)
    win = Window(*[max(0, v) for v in win.flatten()])
    print(type(src.window_transform(win)))
    vrt_options = {
        "crs": src.crs.to_epsg(),
        "transform": src.window_transform(win),
        "width": int(win.width),
        "height": int(win.height),
    }
    
vrt_options.update({'resampling':Resampling.bilinear})
vrt_options

<class 'affine.Affine'>


{'crs': 4326,
 'transform': Affine(0.0083333333333333, 0.0, 91.5,
        0.0, -0.0083333333333333, 29.0),
 'width': 1200,
 'height': 2400,
 'resampling': <Resampling.bilinear: 1>}

In [15]:
# https://rasterio.readthedocs.io/en/latest/topics/virtual-warping.html

outpath = dirpath + outfile

with rasterio.open(inpath) as src:
    with WarpedVRT(src, **vrt_options) as vrt:
        data = vrt.read()
        for _, window in vrt.block_windows():
            data = vrt.read(window=window)
        rio_shutil.copy(vrt, outpath, driver='GTiff')

In [16]:
check_vals_coord(inpath, coords)

[8932, 0, 2599, 286, 12012, 3182]

In [17]:
check_vals_coord(outpath, coords)

[8932, 0, 2599, 286, 12012, 3182]